# ESA CCI Sea Ice Version 4 - Envisat L2P Data Download

## Dataset Information
- **Product**: ESA Sea Ice Climate Change Initiative (Sea_Ice_cci)
- **Description**: Southern hemisphere sea ice thickness from Envisat on the satellite swath (L2P), v4.0
- **FTP Source**: `ftp://ftp.awi.de/sea_ice/projects/cci/crdp/v4p0/l2p_release/sh/envisat/`
- **Time Range**: 2002 - 2012
- **Local Destination**: `D:\phd\data\CCI_v4\l2p\ENV`

## Features
- Multi-threaded concurrent downloads for maximum speed
- Real-time memory monitoring
- Progress tracking with ETA
- Automatic retry on failure
- Resume capability for interrupted downloads

In [1]:
import os
import sys
import ftplib
import psutil
import threading
import time
import hashlib
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from collections import deque
from dataclasses import dataclass, field
from typing import List, Optional, Tuple
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

print("✓ Libraries imported successfully")
print(f"Python version: {sys.version}")
print(f"Available CPU cores: {os.cpu_count()}")

✓ Libraries imported successfully
Python version: 3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]
Available CPU cores: 12


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

@dataclass
class DownloadConfig:
    """Configuration for the FTP download manager."""
    
    # FTP Settings
    ftp_host: str = "ftp.awi.de"
    ftp_base_path: str = "/sea_ice/projects/cci/crdp/v4p0/l2p_release/sh/envisat"
    ftp_timeout: int = 60  # seconds
    
    # Local Settings
    local_base_dir: str = r"D:\phd\data\CCI_v4\l2p\ENV"
    
    # Date Range (Updated: Start from October 17th, 2007)
    start_year: int = 2007
    start_month: int = 10
    start_day: int = 17
    end_year: int = 2012
    end_month: int = 12
    end_day: int = 31
    
    # Performance Settings
    max_workers: int = 8  # Concurrent download threads
    max_retries: int = 3
    retry_delay: int = 5  # seconds
    chunk_size: int = 1024 * 1024  # 1MB chunks for download
    
    # Memory Thresholds (percentage)
    memory_warning_threshold: float = 80.0
    memory_critical_threshold: float = 90.0
    
    # Connection Pool
    connection_pool_size: int = 10
    
    def get_start_date(self) -> datetime:
        """Get the start date as datetime object."""
        return datetime(self.start_year, self.start_month, self.start_day)
    
    def get_end_date(self) -> datetime:
        """Get the end date as datetime object."""
        return datetime(self.end_year, self.end_month, self.end_day)

config = DownloadConfig()

print("=" * 60)
print("DOWNLOAD CONFIGURATION")
print("=" * 60)
print(f"FTP Host: {config.ftp_host}")
print(f"FTP Path: {config.ftp_base_path}")
print(f"Local Directory: {config.local_base_dir}")
print(f"Date Range: {config.get_start_date().strftime('%Y-%m-%d')} to {config.get_end_date().strftime('%Y-%m-%d')}")
print(f"Concurrent Workers: {config.max_workers}")
print(f"Max Retries: {config.max_retries}")
print("=" * 60)

DOWNLOAD CONFIGURATION
FTP Host: ftp.awi.de
FTP Path: /sea_ice/projects/cci/crdp/v4p0/l2p_release/sh/envisat
Local Directory: D:\phd\data\CCI_v4\l2p\ENV
Date Range: 2007-10-17 to 2012-12-31
Concurrent Workers: 8
Max Retries: 3


In [3]:
class MemoryMonitor:
    """Real-time memory monitoring utility."""
    
    def __init__(self, config: DownloadConfig):
        self.config = config
        self._stop_event = threading.Event()
        self._monitor_thread: Optional[threading.Thread] = None
        self._lock = threading.Lock()
        self._history = deque(maxlen=100)
        
    def get_memory_stats(self) -> dict:
        """Get current memory statistics."""
        mem = psutil.virtual_memory()
        return {
            'total_gb': mem.total / (1024 ** 3),
            'available_gb': mem.available / (1024 ** 3),
            'used_gb': mem.used / (1024 ** 3),
            'percent_used': mem.percent,
            'percent_available': 100 - mem.percent
        }
    
    def print_memory_status(self, prefix: str = ""):
        """Print current memory status."""
        stats = self.get_memory_stats()
        status_icon = "✓" if stats['percent_used'] < self.config.memory_warning_threshold else \
                     "⚠" if stats['percent_used'] < self.config.memory_critical_threshold else "🔴"
        
        print(f"{prefix}{status_icon} Memory: {stats['used_gb']:.1f}GB used / "
              f"{stats['total_gb']:.1f}GB total ({stats['percent_used']:.1f}% used) | "
              f"Available: {stats['available_gb']:.1f}GB ({stats['percent_available']:.1f}%)")
        
        return stats
    
    def _monitor_loop(self, interval: int = 30):
        """Background monitoring loop."""
        while not self._stop_event.is_set():
            stats = self.get_memory_stats()
            with self._lock:
                self._history.append({
                    'timestamp': datetime.now(),
                    **stats
                })
            
            if stats['percent_used'] >= self.config.memory_critical_threshold:
                logger.warning(f"🔴 CRITICAL: Memory usage at {stats['percent_used']:.1f}%!")
            elif stats['percent_used'] >= self.config.memory_warning_threshold:
                logger.warning(f"⚠ WARNING: Memory usage at {stats['percent_used']:.1f}%")
                
            self._stop_event.wait(interval)
    
    def start_monitoring(self, interval: int = 30):
        """Start background memory monitoring."""
        self._stop_event.clear()
        self._monitor_thread = threading.Thread(
            target=self._monitor_loop, 
            args=(interval,),
            daemon=True
        )
        self._monitor_thread.start()
        logger.info("Memory monitoring started")
        
    def stop_monitoring(self):
        """Stop background memory monitoring."""
        self._stop_event.set()
        if self._monitor_thread:
            self._monitor_thread.join(timeout=5)
        logger.info("Memory monitoring stopped")

# Initialize memory monitor
memory_monitor = MemoryMonitor(config)
print("Memory Monitor initialized")
memory_monitor.print_memory_status("Current Status: ")

Memory Monitor initialized
Current Status: ✓ Memory: 10.7GB used / 15.4GB total (69.1% used) | Available: 4.8GB (30.9%)


{'total_gb': 15.441062927246094,
 'available_gb': 4.771633148193359,
 'used_gb': 10.669429779052734,
 'percent_used': 69.1,
 'percent_available': 30.900000000000006}

In [4]:
class FTPConnectionPool:
    """Thread-safe FTP connection pool for efficient concurrent downloads."""
    
    def __init__(self, host: str, base_path: str, pool_size: int = 10, timeout: int = 60):
        self.host = host
        self.base_path = base_path
        self.pool_size = pool_size
        self.timeout = timeout
        self._pool: deque = deque()
        self._lock = threading.Lock()
        self._created_count = 0
        
    def _create_connection(self) -> ftplib.FTP:
        """Create a new FTP connection."""
        ftp = ftplib.FTP(timeout=self.timeout)
        ftp.connect(self.host)
        ftp.login()  # Anonymous login
        ftp.cwd(self.base_path)
        return ftp
    
    def get_connection(self) -> ftplib.FTP:
        """Get a connection from the pool or create a new one."""
        with self._lock:
            if self._pool:
                ftp = self._pool.popleft()
                # Test if connection is still alive
                try:
                    ftp.voidcmd("NOOP")
                    return ftp
                except:
                    pass  # Connection dead, create new one
            
            if self._created_count < self.pool_size:
                self._created_count += 1
                return self._create_connection()
        
        # Pool exhausted, create temporary connection
        return self._create_connection()
    
    def return_connection(self, ftp: ftplib.FTP):
        """Return a connection to the pool."""
        try:
            ftp.voidcmd("NOOP")  # Check if still alive
            with self._lock:
                if len(self._pool) < self.pool_size:
                    self._pool.append(ftp)
                    return
        except:
            pass
        
        # Connection dead or pool full, close it
        try:
            ftp.quit()
        except:
            pass
    
    def close_all(self):
        """Close all connections in the pool."""
        with self._lock:
            while self._pool:
                ftp = self._pool.popleft()
                try:
                    ftp.quit()
                except:
                    pass
            self._created_count = 0

print("FTP Connection Pool class defined")

FTP Connection Pool class defined


In [5]:
@dataclass
class FileInfo:
    """Information about a file to download."""
    remote_path: str
    local_path: str
    size: int = 0
    year: int = 0
    month: int = 0
    
@dataclass
class DownloadStats:
    """Statistics for download progress."""
    total_files: int = 0
    completed_files: int = 0
    failed_files: int = 0
    skipped_files: int = 0
    total_bytes: int = 0
    downloaded_bytes: int = 0
    start_time: datetime = field(default_factory=datetime.now)
    failed_list: List[str] = field(default_factory=list)
    
    def get_progress_percent(self) -> float:
        if self.total_files == 0:
            return 0.0
        return (self.completed_files + self.skipped_files) / self.total_files * 100
    
    def get_speed_mbps(self) -> float:
        elapsed = (datetime.now() - self.start_time).total_seconds()
        if elapsed == 0:
            return 0.0
        return (self.downloaded_bytes / (1024 * 1024)) / elapsed
    
    def get_eta(self) -> str:
        if self.completed_files == 0:
            return "Calculating..."
        elapsed = (datetime.now() - self.start_time).total_seconds()
        avg_time_per_file = elapsed / (self.completed_files + self.skipped_files)
        remaining_files = self.total_files - self.completed_files - self.skipped_files
        eta_seconds = avg_time_per_file * remaining_files
        return str(timedelta(seconds=int(eta_seconds)))


class CCIDownloadManager:
    """Manager for downloading CCI Envisat datasets."""
    
    def __init__(self, config: DownloadConfig):
        self.config = config
        self.connection_pool: Optional[FTPConnectionPool] = None
        self.stats = DownloadStats()
        self._stats_lock = threading.Lock()
        
    def connect(self):
        """Initialize connection pool."""
        logger.info(f"Connecting to {self.config.ftp_host}...")
        self.connection_pool = FTPConnectionPool(
            host=self.config.ftp_host,
            base_path=self.config.ftp_base_path,
            pool_size=self.config.connection_pool_size,
            timeout=self.config.ftp_timeout
        )
        # Test connection
        ftp = self.connection_pool.get_connection()
        self.connection_pool.return_connection(ftp)
        logger.info("✓ Connected successfully!")
        
    def disconnect(self):
        """Close all connections."""
        if self.connection_pool:
            self.connection_pool.close_all()
            logger.info("All connections closed")
    
    def _extract_date_from_filename(self, filename: str) -> Optional[datetime]:
        """Extract date from filename like ESACCI-SEAICE-L2P-SITHICK-RA2_ENVISAT-SH-20071017-fv4p0.nc"""
        try:
            # Find the 8-digit date pattern (YYYYMMDD)
            import re
            match = re.search(r'-(\d{8})-', filename)
            if match:
                date_str = match.group(1)
                return datetime.strptime(date_str, '%Y%m%d')
        except:
            pass
        return None
    
    def _is_file_in_date_range(self, filename: str) -> bool:
        """Check if file date is within configured date range."""
        file_date = self._extract_date_from_filename(filename)
        if file_date is None:
            return False
        
        start_date = self.config.get_start_date()
        end_date = self.config.get_end_date()
        
        return start_date <= file_date <= end_date
    
    def discover_files(self) -> List[FileInfo]:
        """Discover all files to download within the specified date range."""
        logger.info("Discovering files on FTP server...")
        logger.info(f"Date filter: {self.config.get_start_date().strftime('%Y-%m-%d')} to {self.config.get_end_date().strftime('%Y-%m-%d')}")
        files = []
        
        ftp = self.connection_pool.get_connection()
        try:
            for year in range(self.config.start_year, self.config.end_year + 1):
                year_path = str(year)
                
                try:
                    ftp.cwd(f"{self.config.ftp_base_path}/{year_path}")
                except ftplib.error_perm:
                    logger.warning(f"Year {year} directory not found, skipping...")
                    continue
                
                # List months
                months = []
                ftp.retrlines('NLST', months.append)
                
                for month in sorted(months):
                    try:
                        month_int = int(month)
                    except ValueError:
                        continue
                    
                    # Quick filter: skip months entirely before start date
                    if year == self.config.start_year and month_int < self.config.start_month:
                        continue
                    # Quick filter: skip months entirely after end date
                    if year == self.config.end_year and month_int > self.config.end_month:
                        continue
                        
                    month_path = f"{year_path}/{month}"
                    
                    try:
                        ftp.cwd(f"{self.config.ftp_base_path}/{month_path}")
                    except ftplib.error_perm:
                        continue
                    
                    # List files in month
                    daily_files = []
                    ftp.retrlines('NLST', daily_files.append)
                    
                    files_in_month = 0
                    for filename in daily_files:
                        if filename.endswith('.nc'):
                            # Apply date filter
                            if not self._is_file_in_date_range(filename):
                                continue
                            
                            remote_path = f"{month_path}/{filename}"
                            local_path = os.path.join(
                                self.config.local_base_dir,
                                str(year),
                                month,
                                filename
                            )
                            
                            # Get file size
                            try:
                                size = ftp.size(filename)
                            except Exception:
                                size = 0
                            
                            files.append(FileInfo(
                                remote_path=remote_path,
                                local_path=local_path,
                                size=size or 0,
                                year=year,
                                month=month_int
                            ))
                            files_in_month += 1
                    
                    if files_in_month > 0:
                        logger.info(f"  Found {files_in_month} files in {year}/{month}")
                    
        finally:
            self.connection_pool.return_connection(ftp)
        
        logger.info(f"✓ Total files discovered: {len(files)}")
        total_size_gb = sum(f.size for f in files) / (1024**3)
        logger.info(f"✓ Total size: {total_size_gb:.2f} GB")
        
        return files
    
    def _download_file(self, file_info: FileInfo) -> Tuple[bool, str]:
        """Download a single file with retry logic."""
        # Check if file already exists and is complete
        if os.path.exists(file_info.local_path):
            local_size = os.path.getsize(file_info.local_path)
            if file_info.size > 0 and local_size >= file_info.size:
                return True, "skipped"
        
        # Create directory if needed
        os.makedirs(os.path.dirname(file_info.local_path), exist_ok=True)
        
        for attempt in range(self.config.max_retries):
            ftp = None
            try:
                ftp = self.connection_pool.get_connection()
                
                # Change to the correct directory
                ftp.cwd(self.config.ftp_base_path)
                
                # Download file
                with open(file_info.local_path, 'wb') as f:
                    ftp.retrbinary(
                        f'RETR {file_info.remote_path}',
                        f.write,
                        blocksize=self.config.chunk_size
                    )
                
                self.connection_pool.return_connection(ftp)
                return True, "downloaded"
                
            except Exception as e:
                if ftp:
                    try:
                        ftp.quit()
                    except:
                        pass
                
                if attempt < self.config.max_retries - 1:
                    time.sleep(self.config.retry_delay)
                else:
                    # Clean up partial file
                    if os.path.exists(file_info.local_path):
                        try:
                            os.remove(file_info.local_path)
                        except:
                            pass
                    return False, str(e)
        
        return False, "Max retries exceeded"
    
    def _update_stats(self, file_info: FileInfo, success: bool, status: str):
        """Thread-safe stats update."""
        with self._stats_lock:
            if success:
                if status == "skipped":
                    self.stats.skipped_files += 1
                else:
                    self.stats.completed_files += 1
                    self.stats.downloaded_bytes += file_info.size
            else:
                self.stats.failed_files += 1
                self.stats.failed_list.append(file_info.remote_path)
    
    def download_all(self, files: List[FileInfo], show_progress_interval: int = 10):
        """Download all files using thread pool."""
        self.stats = DownloadStats(
            total_files=len(files),
            total_bytes=sum(f.size for f in files),
            start_time=datetime.now()
        )
        
        logger.info(f"Starting download of {len(files)} files with {self.config.max_workers} workers...")
        
        last_progress_time = time.time()
        
        with ThreadPoolExecutor(max_workers=self.config.max_workers) as executor:
            # Submit all download tasks
            future_to_file = {
                executor.submit(self._download_file, f): f 
                for f in files
            }
            
            for future in as_completed(future_to_file):
                file_info = future_to_file[future]
                try:
                    success, status = future.result()
                    self._update_stats(file_info, success, status)
                except Exception as e:
                    self._update_stats(file_info, False, str(e))
                
                # Progress report
                current_time = time.time()
                if current_time - last_progress_time >= show_progress_interval:
                    self._print_progress()
                    last_progress_time = current_time
        
        self._print_final_stats()
    
    def _print_progress(self):
        """Print current progress."""
        with self._stats_lock:
            progress = self.stats.get_progress_percent()
            speed = self.stats.get_speed_mbps()
            eta = self.stats.get_eta()
            
            print(f"\r📊 Progress: {progress:.1f}% | "
                  f"Files: {self.stats.completed_files + self.stats.skipped_files}/{self.stats.total_files} | "
                  f"Speed: {speed:.2f} MB/s | "
                  f"ETA: {eta} | "
                  f"Skipped: {self.stats.skipped_files} | "
                  f"Failed: {self.stats.failed_files}", end="")
            print()  # New line for notebook
    
    def _print_final_stats(self):
        """Print final download statistics."""
        elapsed = (datetime.now() - self.stats.start_time).total_seconds()
        
        print("\n" + "=" * 60)
        print("DOWNLOAD COMPLETE")
        print("=" * 60)
        print(f"✓ Total files processed: {self.stats.total_files}")
        print(f"✓ Successfully downloaded: {self.stats.completed_files}")
        print(f"✓ Skipped (already exists): {self.stats.skipped_files}")
        print(f"✗ Failed: {self.stats.failed_files}")
        print(f"📦 Total data downloaded: {self.stats.downloaded_bytes / (1024**3):.2f} GB")
        print(f"⏱ Total time: {timedelta(seconds=int(elapsed))}")
        print(f"🚀 Average speed: {self.stats.get_speed_mbps():.2f} MB/s")
        print("=" * 60)
        
        if self.stats.failed_list:
            print("\n⚠ Failed files:")
            for f in self.stats.failed_list[:10]:
                print(f"  - {f}")
            if len(self.stats.failed_list) > 10:
                print(f"  ... and {len(self.stats.failed_list) - 10} more")

print("Download Manager class defined")

Download Manager class defined


In [6]:
# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main execution function."""
    
    print("=" * 60)
    print("CCI V4 ENVISAT L2P DATA DOWNLOAD")
    print("=" * 60)
    print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # Check and create local directory
    local_dir = Path(config.local_base_dir)
    if not local_dir.exists():
        local_dir.mkdir(parents=True, exist_ok=True)
        print(f"✓ Created directory: {local_dir}")
    else:
        print(f"✓ Directory exists: {local_dir}")
    
    # Initial memory check
    print("\n--- Initial Memory Status ---")
    memory_monitor.print_memory_status()
    
    # Start memory monitoring
    memory_monitor.start_monitoring(interval=60)
    
    # Initialize download manager
    manager = CCIDownloadManager(config)
    
    try:
        # Connect to FTP
        manager.connect()
        
        # Discover files
        print("\n--- File Discovery ---")
        files = manager.discover_files()
        
        if not files:
            print("No files found to download!")
            return
        
        # Confirm download
        total_size_gb = sum(f.size for f in files) / (1024**3)
        print(f"\n📁 Ready to download {len(files)} files ({total_size_gb:.2f} GB)")
        print(f"📂 Destination: {config.local_base_dir}")
        
        # Memory check before download
        print("\n--- Pre-Download Memory Status ---")
        stats = memory_monitor.print_memory_status()
        
        if stats['percent_used'] > config.memory_critical_threshold:
            print("⚠ WARNING: Memory usage is critical! Consider freeing memory before proceeding.")
        
        # Start download
        print("\n--- Starting Download ---")
        manager.download_all(files)
        
        # Final memory status
        print("\n--- Final Memory Status ---")
        memory_monitor.print_memory_status()
        
    except KeyboardInterrupt:
        print("\n\n⚠ Download interrupted by user!")
    except Exception as e:
        logger.error(f"Error during download: {e}")
        raise
    finally:
        # Cleanup
        manager.disconnect()
        memory_monitor.stop_monitoring()
        print(f"\nEnd Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Run the download
if __name__ == "__main__" or True:  # True for notebook execution
    main()

2025-12-07 12:49:44 - INFO - Memory monitoring started
2025-12-07 12:49:44 - INFO - Connecting to ftp.awi.de...


CCI V4 ENVISAT L2P DATA DOWNLOAD
Start Time: 2025-12-07 12:49:44

✓ Directory exists: D:\phd\data\CCI_v4\l2p\ENV

--- Initial Memory Status ---
✓ Memory: 10.6GB used / 15.4GB total (68.8% used) | Available: 4.8GB (31.2%)


2025-12-07 12:49:46 - INFO - ✓ Connected successfully!
2025-12-07 12:49:46 - INFO - Discovering files on FTP server...
2025-12-07 12:49:46 - INFO - Date filter: 2007-10-17 to 2012-12-31



--- File Discovery ---


2025-12-07 12:49:54 - INFO -   Found 15 files in 2007/10
2025-12-07 12:50:04 - INFO -   Found 30 files in 2007/11
2025-12-07 12:50:15 - INFO -   Found 31 files in 2007/12
2025-12-07 12:50:27 - INFO -   Found 31 files in 2008/01
2025-12-07 12:50:37 - INFO -   Found 29 files in 2008/02
2025-12-07 12:50:48 - INFO -   Found 31 files in 2008/03
2025-12-07 12:50:58 - INFO -   Found 30 files in 2008/04
2025-12-07 12:51:09 - INFO -   Found 31 files in 2008/05
2025-12-07 12:51:19 - INFO -   Found 30 files in 2008/06
2025-12-07 12:51:30 - INFO -   Found 31 files in 2008/07
2025-12-07 12:51:40 - INFO -   Found 31 files in 2008/08
2025-12-07 12:51:51 - INFO -   Found 30 files in 2008/09
2025-12-07 12:52:02 - INFO -   Found 31 files in 2008/10
2025-12-07 12:52:12 - INFO -   Found 30 files in 2008/11
2025-12-07 12:52:23 - INFO -   Found 31 files in 2008/12
2025-12-07 12:52:35 - INFO -   Found 31 files in 2009/01
2025-12-07 12:52:45 - INFO -   Found 28 files in 2009/02
2025-12-07 12:52:55 - INFO -   


📁 Ready to download 1626 files (1.98 GB)
📂 Destination: D:\phd\data\CCI_v4\l2p\ENV

--- Pre-Download Memory Status ---
✓ Memory: 9.9GB used / 15.4GB total (64.3% used) | Available: 5.5GB (35.7%)

--- Starting Download ---
📊 Progress: 0.4% | Files: 7/1626 | Speed: 1.16 MB/s | ETA: 0:42:46 | Skipped: 0 | Failed: 0
📊 Progress: 1.5% | Files: 25/1626 | Speed: 2.03 MB/s | ETA: 0:22:45 | Skipped: 0 | Failed: 0
📊 Progress: 2.9% | Files: 47/1626 | Speed: 2.31 MB/s | ETA: 0:17:50 | Skipped: 0 | Failed: 0
📊 Progress: 4.1% | Files: 66/1626 | Speed: 2.20 MB/s | ETA: 0:16:30 | Skipped: 0 | Failed: 0
📊 Progress: 5.4% | Files: 88/1626 | Speed: 2.03 MB/s | ETA: 0:15:14 | Skipped: 0 | Failed: 0
📊 Progress: 6.9% | Files: 112/1626 | Speed: 1.86 MB/s | ETA: 0:14:05 | Skipped: 0 | Failed: 0
📊 Progress: 8.2% | Files: 134/1626 | Speed: 1.70 MB/s | ETA: 0:13:30 | Skipped: 0 | Failed: 0
📊 Progress: 9.6% | Files: 156/1626 | Speed: 1.59 MB/s | ETA: 0:13:06 | Skipped: 0 | Failed: 0
📊 Progress: 10.9% | Files: 178/

2025-12-07 13:13:07 - INFO - All connections closed
2025-12-07 13:13:07 - INFO - Memory monitoring stopped



End Time: 2025-12-07 13:13:07


# ESA CCI Sea Ice Version 4 - CryoSat-2 L2P Data Download

## Dataset Information
- **Product**: ESA Sea Ice Climate Change Initiative (Sea_Ice_cci)
- **Description**: Southern hemisphere sea ice thickness from CryoSat-2 on the satellite swath (L2P), v4.0
- **FTP Source**: `ftp://ftp.awi.de/sea_ice/projects/cci/crdp/v4p0/l2p_release/sh/cryosat2/`
- **Time Range**: 2010 - 2024
- **Local Destination**: `D:\phd\data\CCI_v4\l2p\CS2`
- **File Pattern**: `ESACCI-SEAICE-L2P-SITHICK-SIRAL_CRYOSAT2-SH-YYYYMMDD-fv4p0.nc`

In [7]:
# =============================================================================
# CRYOSAT-2 CONFIGURATION
# =============================================================================

@dataclass
class CS2DownloadConfig:
    """Configuration for the CryoSat-2 FTP download manager."""
    
    # FTP Settings
    ftp_host: str = "ftp.awi.de"
    ftp_base_path: str = "/sea_ice/projects/cci/crdp/v4p0/l2p_release/sh/cryosat2"
    ftp_timeout: int = 60  # seconds
    
    # Local Settings
    local_base_dir: str = r"D:\phd\data\CCI_v4\l2p\CS2"
    
    # Date Range: Full CryoSat-2 record (2010 - 2024)
    start_year: int = 2010
    start_month: int = 1
    start_day: int = 1
    end_year: int = 2024
    end_month: int = 12
    end_day: int = 31
    
    # Performance Settings
    max_workers: int = 8  # Concurrent download threads
    max_retries: int = 3
    retry_delay: int = 5  # seconds
    chunk_size: int = 1024 * 1024  # 1MB chunks for download
    
    # Memory Thresholds (percentage)
    memory_warning_threshold: float = 80.0
    memory_critical_threshold: float = 90.0
    
    # Connection Pool
    connection_pool_size: int = 10
    
    def get_start_date(self) -> datetime:
        """Get the start date as datetime object."""
        return datetime(self.start_year, self.start_month, self.start_day)
    
    def get_end_date(self) -> datetime:
        """Get the end date as datetime object."""
        return datetime(self.end_year, self.end_month, self.end_day)

cs2_config = CS2DownloadConfig()

print("=" * 60)
print("CRYOSAT-2 DOWNLOAD CONFIGURATION")
print("=" * 60)
print(f"FTP Host: {cs2_config.ftp_host}")
print(f"FTP Path: {cs2_config.ftp_base_path}")
print(f"Local Directory: {cs2_config.local_base_dir}")
print(f"Date Range: {cs2_config.get_start_date().strftime('%Y-%m-%d')} to {cs2_config.get_end_date().strftime('%Y-%m-%d')}")
print(f"Concurrent Workers: {cs2_config.max_workers}")
print(f"Max Retries: {cs2_config.max_retries}")
print("=" * 60)

CRYOSAT-2 DOWNLOAD CONFIGURATION
FTP Host: ftp.awi.de
FTP Path: /sea_ice/projects/cci/crdp/v4p0/l2p_release/sh/cryosat2
Local Directory: D:\phd\data\CCI_v4\l2p\CS2
Date Range: 2010-01-01 to 2024-12-31
Concurrent Workers: 8
Max Retries: 3


In [8]:
class CS2DownloadManager:
    """Manager for downloading CCI CryoSat-2 datasets."""
    
    def __init__(self, config: CS2DownloadConfig):
        self.config = config
        self.connection_pool: Optional[FTPConnectionPool] = None
        self.stats = DownloadStats()
        self._stats_lock = threading.Lock()
        
    def connect(self):
        """Initialize connection pool."""
        logger.info(f"Connecting to {self.config.ftp_host}...")
        self.connection_pool = FTPConnectionPool(
            host=self.config.ftp_host,
            base_path=self.config.ftp_base_path,
            pool_size=self.config.connection_pool_size,
            timeout=self.config.ftp_timeout
        )
        # Test connection
        ftp = self.connection_pool.get_connection()
        self.connection_pool.return_connection(ftp)
        logger.info("✓ Connected successfully to CryoSat-2 directory!")
        
    def disconnect(self):
        """Close all connections."""
        if self.connection_pool:
            self.connection_pool.close_all()
            logger.info("All connections closed")
    
    def _extract_date_from_filename(self, filename: str) -> Optional[datetime]:
        """Extract date from filename like ESACCI-SEAICE-L2P-SITHICK-SIRAL_CRYOSAT2-SH-20101101-fv4p0.nc"""
        try:
            import re
            match = re.search(r'-(\d{8})-', filename)
            if match:
                date_str = match.group(1)
                return datetime.strptime(date_str, '%Y%m%d')
        except:
            pass
        return None
    
    def _is_file_in_date_range(self, filename: str) -> bool:
        """Check if file date is within configured date range."""
        file_date = self._extract_date_from_filename(filename)
        if file_date is None:
            return False
        
        start_date = self.config.get_start_date()
        end_date = self.config.get_end_date()
        
        return start_date <= file_date <= end_date
    
    def discover_files(self) -> List[FileInfo]:
        """Discover all CryoSat-2 files to download within the specified date range."""
        logger.info("Discovering CryoSat-2 files on FTP server...")
        logger.info(f"Date filter: {self.config.get_start_date().strftime('%Y-%m-%d')} to {self.config.get_end_date().strftime('%Y-%m-%d')}")
        files = []
        
        ftp = self.connection_pool.get_connection()
        try:
            for year in range(self.config.start_year, self.config.end_year + 1):
                year_path = str(year)
                
                try:
                    ftp.cwd(f"{self.config.ftp_base_path}/{year_path}")
                except ftplib.error_perm:
                    logger.warning(f"Year {year} directory not found, skipping...")
                    continue
                
                # List months
                months = []
                ftp.retrlines('NLST', months.append)
                
                for month in sorted(months):
                    try:
                        month_int = int(month)
                    except ValueError:
                        continue
                    
                    # Quick filter: skip months entirely before start date
                    if year == self.config.start_year and month_int < self.config.start_month:
                        continue
                    # Quick filter: skip months entirely after end date
                    if year == self.config.end_year and month_int > self.config.end_month:
                        continue
                        
                    month_path = f"{year_path}/{month}"
                    
                    try:
                        ftp.cwd(f"{self.config.ftp_base_path}/{month_path}")
                    except ftplib.error_perm:
                        continue
                    
                    # List files in month
                    daily_files = []
                    ftp.retrlines('NLST', daily_files.append)
                    
                    files_in_month = 0
                    for filename in daily_files:
                        if filename.endswith('.nc'):
                            # Apply date filter
                            if not self._is_file_in_date_range(filename):
                                continue
                            
                            remote_path = f"{month_path}/{filename}"
                            local_path = os.path.join(
                                self.config.local_base_dir,
                                str(year),
                                month,
                                filename
                            )
                            
                            # Get file size
                            try:
                                size = ftp.size(filename)
                            except Exception:
                                size = 0
                            
                            files.append(FileInfo(
                                remote_path=remote_path,
                                local_path=local_path,
                                size=size or 0,
                                year=year,
                                month=month_int
                            ))
                            files_in_month += 1
                    
                    if files_in_month > 0:
                        logger.info(f"  Found {files_in_month} files in {year}/{month}")
                    
        finally:
            self.connection_pool.return_connection(ftp)
        
        logger.info(f"✓ Total CryoSat-2 files discovered: {len(files)}")
        total_size_gb = sum(f.size for f in files) / (1024**3)
        logger.info(f"✓ Total size: {total_size_gb:.2f} GB")
        
        return files
    
    def _download_file(self, file_info: FileInfo) -> Tuple[bool, str]:
        """Download a single file with retry logic."""
        # Check if file already exists and is complete
        if os.path.exists(file_info.local_path):
            local_size = os.path.getsize(file_info.local_path)
            if file_info.size > 0 and local_size >= file_info.size:
                return True, "skipped"
        
        # Create directory if needed
        os.makedirs(os.path.dirname(file_info.local_path), exist_ok=True)
        
        for attempt in range(self.config.max_retries):
            ftp = None
            try:
                ftp = self.connection_pool.get_connection()
                
                # Change to the correct directory
                ftp.cwd(self.config.ftp_base_path)
                
                # Download file
                with open(file_info.local_path, 'wb') as f:
                    ftp.retrbinary(
                        f'RETR {file_info.remote_path}',
                        f.write,
                        blocksize=self.config.chunk_size
                    )
                
                self.connection_pool.return_connection(ftp)
                return True, "downloaded"
                
            except Exception as e:
                if ftp:
                    try:
                        ftp.quit()
                    except:
                        pass
                
                if attempt < self.config.max_retries - 1:
                    time.sleep(self.config.retry_delay)
                else:
                    # Clean up partial file
                    if os.path.exists(file_info.local_path):
                        try:
                            os.remove(file_info.local_path)
                        except:
                            pass
                    return False, str(e)
        
        return False, "Max retries exceeded"
    
    def _update_stats(self, file_info: FileInfo, success: bool, status: str):
        """Thread-safe stats update."""
        with self._stats_lock:
            if success:
                if status == "skipped":
                    self.stats.skipped_files += 1
                else:
                    self.stats.completed_files += 1
                    self.stats.downloaded_bytes += file_info.size
            else:
                self.stats.failed_files += 1
                self.stats.failed_list.append(file_info.remote_path)
    
    def download_all(self, files: List[FileInfo], show_progress_interval: int = 10):
        """Download all files using thread pool."""
        self.stats = DownloadStats(
            total_files=len(files),
            total_bytes=sum(f.size for f in files),
            start_time=datetime.now()
        )
        
        logger.info(f"Starting download of {len(files)} CryoSat-2 files with {self.config.max_workers} workers...")
        
        last_progress_time = time.time()
        
        with ThreadPoolExecutor(max_workers=self.config.max_workers) as executor:
            # Submit all download tasks
            future_to_file = {
                executor.submit(self._download_file, f): f 
                for f in files
            }
            
            for future in as_completed(future_to_file):
                file_info = future_to_file[future]
                try:
                    success, status = future.result()
                    self._update_stats(file_info, success, status)
                except Exception as e:
                    self._update_stats(file_info, False, str(e))
                
                # Progress report
                current_time = time.time()
                if current_time - last_progress_time >= show_progress_interval:
                    self._print_progress()
                    last_progress_time = current_time
        
        self._print_final_stats()
    
    def _print_progress(self):
        """Print current progress."""
        with self._stats_lock:
            progress = self.stats.get_progress_percent()
            speed = self.stats.get_speed_mbps()
            eta = self.stats.get_eta()
            
            print(f"\r📊 Progress: {progress:.1f}% | "
                  f"Files: {self.stats.completed_files + self.stats.skipped_files}/{self.stats.total_files} | "
                  f"Speed: {speed:.2f} MB/s | "
                  f"ETA: {eta} | "
                  f"Skipped: {self.stats.skipped_files} | "
                  f"Failed: {self.stats.failed_files}", end="")
            print()  # New line for notebook
    
    def _print_final_stats(self):
        """Print final download statistics."""
        elapsed = (datetime.now() - self.stats.start_time).total_seconds()
        
        print("\n" + "=" * 60)
        print("CRYOSAT-2 DOWNLOAD COMPLETE")
        print("=" * 60)
        print(f"✓ Total files processed: {self.stats.total_files}")
        print(f"✓ Successfully downloaded: {self.stats.completed_files}")
        print(f"✓ Skipped (already exists): {self.stats.skipped_files}")
        print(f"✗ Failed: {self.stats.failed_files}")
        print(f"📦 Total data downloaded: {self.stats.downloaded_bytes / (1024**3):.2f} GB")
        print(f"⏱ Total time: {timedelta(seconds=int(elapsed))}")
        print(f"🚀 Average speed: {self.stats.get_speed_mbps():.2f} MB/s")
        print("=" * 60)
        
        if self.stats.failed_list:
            print("\n⚠ Failed files:")
            for f in self.stats.failed_list[:10]:
                print(f"  - {f}")
            if len(self.stats.failed_list) > 10:
                print(f"  ... and {len(self.stats.failed_list) - 10} more")

print("CryoSat-2 Download Manager class defined")

CryoSat-2 Download Manager class defined


In [9]:
# =============================================================================
# CRYOSAT-2 MAIN EXECUTION
# =============================================================================

def main_cs2():
    """Main execution function for CryoSat-2 downloads."""
    
    print("=" * 60)
    print("CCI V4 CRYOSAT-2 L2P DATA DOWNLOAD")
    print("=" * 60)
    print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # Check and create local directory
    local_dir = Path(cs2_config.local_base_dir)
    if not local_dir.exists():
        local_dir.mkdir(parents=True, exist_ok=True)
        print(f"✓ Created directory: {local_dir}")
    else:
        print(f"✓ Directory exists: {local_dir}")
    
    # Initial memory check
    print("\n--- Initial Memory Status ---")
    memory_monitor.print_memory_status()
    
    # Start memory monitoring
    memory_monitor.start_monitoring(interval=60)
    
    # Initialize download manager
    manager = CS2DownloadManager(cs2_config)
    
    try:
        # Connect to FTP
        manager.connect()
        
        # Discover files
        print("\n--- File Discovery ---")
        files = manager.discover_files()
        
        if not files:
            print("No CryoSat-2 files found to download!")
            return
        
        # Confirm download
        total_size_gb = sum(f.size for f in files) / (1024**3)
        print(f"\n📁 Ready to download {len(files)} CryoSat-2 files ({total_size_gb:.2f} GB)")
        print(f"📂 Destination: {cs2_config.local_base_dir}")
        
        # Memory check before download
        print("\n--- Pre-Download Memory Status ---")
        stats = memory_monitor.print_memory_status()
        
        if stats['percent_used'] > cs2_config.memory_critical_threshold:
            print("⚠ WARNING: Memory usage is critical! Consider freeing memory before proceeding.")
        
        # Start download
        print("\n--- Starting CryoSat-2 Download ---")
        manager.download_all(files)
        
        # Final memory status
        print("\n--- Final Memory Status ---")
        memory_monitor.print_memory_status()
        
    except KeyboardInterrupt:
        print("\n\n⚠ Download interrupted by user!")
    except Exception as e:
        logger.error(f"Error during CryoSat-2 download: {e}")
        raise
    finally:
        # Cleanup
        manager.disconnect()
        memory_monitor.stop_monitoring()
        print(f"\nEnd Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Run the CryoSat-2 download
if __name__ == "__main__" or True:  # True for notebook execution
    main_cs2()

2025-12-07 13:21:53 - INFO - Memory monitoring started
2025-12-07 13:21:53 - INFO - Connecting to ftp.awi.de...


CCI V4 CRYOSAT-2 L2P DATA DOWNLOAD
Start Time: 2025-12-07 13:21:53

✓ Directory exists: D:\phd\data\CCI_v4\l2p\CS2

--- Initial Memory Status ---
✓ Memory: 10.5GB used / 15.4GB total (68.1% used) | Available: 4.9GB (31.9%)


2025-12-07 13:21:54 - INFO - ✓ Connected successfully to CryoSat-2 directory!
2025-12-07 13:21:54 - INFO - Discovering CryoSat-2 files on FTP server...
2025-12-07 13:21:54 - INFO - Date filter: 2010-01-01 to 2024-12-31



--- File Discovery ---


2025-12-07 13:22:07 - INFO -   Found 30 files in 2010/11
2025-12-07 13:22:17 - INFO -   Found 31 files in 2010/12
2025-12-07 13:22:29 - INFO -   Found 30 files in 2011/01
2025-12-07 13:22:39 - INFO -   Found 28 files in 2011/02
2025-12-07 13:22:49 - INFO -   Found 31 files in 2011/03
2025-12-07 13:22:59 - INFO -   Found 30 files in 2011/04
2025-12-07 13:23:09 - INFO -   Found 27 files in 2011/05
2025-12-07 13:23:19 - INFO -   Found 30 files in 2011/06
2025-12-07 13:23:29 - INFO -   Found 31 files in 2011/07
2025-12-07 13:23:40 - INFO -   Found 31 files in 2011/08
2025-12-07 13:23:50 - INFO -   Found 29 files in 2011/09
2025-12-07 13:24:00 - INFO -   Found 31 files in 2011/10
2025-12-07 13:24:10 - INFO -   Found 30 files in 2011/11
2025-12-07 13:24:21 - INFO -   Found 31 files in 2011/12
2025-12-07 13:24:33 - INFO -   Found 31 files in 2012/01
2025-12-07 13:24:43 - INFO -   Found 29 files in 2012/02
2025-12-07 13:24:53 - INFO -   Found 31 files in 2012/03
2025-12-07 13:25:03 - INFO -   


📁 Ready to download 4909 CryoSat-2 files (6.81 GB)
📂 Destination: D:\phd\data\CCI_v4\l2p\CS2

--- Pre-Download Memory Status ---
✓ Memory: 10.3GB used / 15.4GB total (66.5% used) | Available: 5.2GB (33.5%)

--- Starting CryoSat-2 Download ---
📊 Progress: 0.1% | Files: 6/4909 | Speed: 1.02 MB/s | ETA: 2:16:50 | Skipped: 0 | Failed: 0
📊 Progress: 0.5% | Files: 24/4909 | Speed: 1.83 MB/s | ETA: 1:09:14 | Skipped: 0 | Failed: 0
📊 Progress: 0.9% | Files: 45/4909 | Speed: 2.08 MB/s | ETA: 0:55:00 | Skipped: 0 | Failed: 0
📊 Progress: 1.3% | Files: 66/4909 | Speed: 1.87 MB/s | ETA: 0:50:31 | Skipped: 0 | Failed: 0
📊 Progress: 1.8% | Files: 90/4909 | Speed: 1.66 MB/s | ETA: 0:46:18 | Skipped: 0 | Failed: 0
📊 Progress: 2.3% | Files: 114/4909 | Speed: 1.51 MB/s | ETA: 0:43:33 | Skipped: 0 | Failed: 0
📊 Progress: 2.8% | Files: 136/4909 | Speed: 1.41 MB/s | ETA: 0:42:19 | Skipped: 0 | Failed: 0
📊 Progress: 3.2% | Files: 159/4909 | Speed: 1.39 MB/s | ETA: 0:41:27 | Skipped: 0 | Failed: 0
📊 Progress

2025-12-07 14:34:06 - INFO - All connections closed
2025-12-07 14:34:06 - INFO - Memory monitoring stopped



End Time: 2025-12-07 14:34:06


# ESA CCI Sea Ice Version 4 - Envisat L3C Data Download

## Dataset Information
- **Product**: ESA Sea Ice Climate Change Initiative (Sea_Ice_cci)
- **Description**: Southern hemisphere sea ice thickness from Envisat on 50km EASE2 grid (L3C), v4.0
- **FTP Source**: `ftp://ftp.awi.de/sea_ice/projects/cci/crdp/v4p0/l3c_release/sh/envisat/`
- **Time Range**: 2002 - 2012
- **Local Destination**: `D:\phd\data\CCI_v4\l3c\ENV`
- **File Pattern**: `ESACCI-SEAICE-L3C-SITHICK-RA2_ENVISAT-SH_50KM_EASE2-YYYYMM-fv4p0.nc`
- **Temporal Resolution**: Monthly

In [10]:
# =============================================================================
# ENVISAT L3C CONFIGURATION
# =============================================================================

@dataclass
class ENVL3CDownloadConfig:
    """Configuration for the Envisat L3C FTP download manager."""
    
    # FTP Settings
    ftp_host: str = "ftp.awi.de"
    ftp_base_path: str = "/sea_ice/projects/cci/crdp/v4p0/l3c_release/sh/envisat"
    ftp_timeout: int = 60  # seconds
    
    # Local Settings
    local_base_dir: str = r"D:\phd\data\CCI_v4\l3c\ENV"
    
    # Date Range: Full Envisat L3C record (2002 - 2012)
    start_year: int = 2002
    start_month: int = 1
    end_year: int = 2012
    end_month: int = 12
    
    # Performance Settings
    max_workers: int = 8  # Concurrent download threads
    max_retries: int = 3
    retry_delay: int = 5  # seconds
    chunk_size: int = 1024 * 1024  # 1MB chunks for download
    
    # Memory Thresholds (percentage)
    memory_warning_threshold: float = 80.0
    memory_critical_threshold: float = 90.0
    
    # Connection Pool
    connection_pool_size: int = 10
    
    def get_start_date(self) -> str:
        """Get the start date as YYYY-MM string."""
        return f"{self.start_year}-{self.start_month:02d}"
    
    def get_end_date(self) -> str:
        """Get the end date as YYYY-MM string."""
        return f"{self.end_year}-{self.end_month:02d}"

env_l3c_config = ENVL3CDownloadConfig()

print("=" * 60)
print("ENVISAT L3C DOWNLOAD CONFIGURATION")
print("=" * 60)
print(f"FTP Host: {env_l3c_config.ftp_host}")
print(f"FTP Path: {env_l3c_config.ftp_base_path}")
print(f"Local Directory: {env_l3c_config.local_base_dir}")
print(f"Date Range: {env_l3c_config.get_start_date()} to {env_l3c_config.get_end_date()}")
print(f"Concurrent Workers: {env_l3c_config.max_workers}")
print(f"Max Retries: {env_l3c_config.max_retries}")
print("=" * 60)

ENVISAT L3C DOWNLOAD CONFIGURATION
FTP Host: ftp.awi.de
FTP Path: /sea_ice/projects/cci/crdp/v4p0/l3c_release/sh/envisat
Local Directory: D:\phd\data\CCI_v4\l3c\ENV
Date Range: 2002-01 to 2012-12
Concurrent Workers: 8
Max Retries: 3


In [11]:
class ENVL3CDownloadManager:
    """Manager for downloading CCI Envisat L3C datasets (monthly gridded)."""
    
    def __init__(self, config: ENVL3CDownloadConfig):
        self.config = config
        self.connection_pool: Optional[FTPConnectionPool] = None
        self.stats = DownloadStats()
        self._stats_lock = threading.Lock()
        
    def connect(self):
        """Initialize connection pool."""
        logger.info(f"Connecting to {self.config.ftp_host}...")
        self.connection_pool = FTPConnectionPool(
            host=self.config.ftp_host,
            base_path=self.config.ftp_base_path,
            pool_size=self.config.connection_pool_size,
            timeout=self.config.ftp_timeout
        )
        # Test connection
        ftp = self.connection_pool.get_connection()
        self.connection_pool.return_connection(ftp)
        logger.info("✓ Connected successfully to Envisat L3C directory!")
        
    def disconnect(self):
        """Close all connections."""
        if self.connection_pool:
            self.connection_pool.close_all()
            logger.info("All connections closed")
    
    def _extract_yearmonth_from_filename(self, filename: str) -> Optional[Tuple[int, int]]:
        """Extract year and month from filename like ESACCI-SEAICE-L3C-SITHICK-RA2_ENVISAT-SH_50KM_EASE2-200210-fv4p0.nc"""
        try:
            import re
            # Match 6-digit YYYYMM pattern before -fv
            match = re.search(r'-(\d{6})-fv', filename)
            if match:
                yyyymm = match.group(1)
                year = int(yyyymm[:4])
                month = int(yyyymm[4:6])
                return (year, month)
        except:
            pass
        return None
    
    def _is_file_in_date_range(self, filename: str) -> bool:
        """Check if file year/month is within configured date range."""
        result = self._extract_yearmonth_from_filename(filename)
        if result is None:
            return False
        
        year, month = result
        
        # Check if within range
        start_val = self.config.start_year * 100 + self.config.start_month
        end_val = self.config.end_year * 100 + self.config.end_month
        file_val = year * 100 + month
        
        return start_val <= file_val <= end_val
    
    def discover_files(self) -> List[FileInfo]:
        """Discover all Envisat L3C files to download within the specified date range."""
        logger.info("Discovering Envisat L3C files on FTP server...")
        logger.info(f"Date filter: {self.config.get_start_date()} to {self.config.get_end_date()}")
        files = []
        
        ftp = self.connection_pool.get_connection()
        try:
            for year in range(self.config.start_year, self.config.end_year + 1):
                year_path = str(year)
                
                try:
                    ftp.cwd(f"{self.config.ftp_base_path}/{year_path}")
                except ftplib.error_perm:
                    logger.warning(f"Year {year} directory not found, skipping...")
                    continue
                
                # List files directly in year directory (L3C has monthly files per year)
                year_files = []
                ftp.retrlines('NLST', year_files.append)
                
                files_in_year = 0
                for filename in year_files:
                    if filename.endswith('.nc'):
                        # Apply date filter
                        if not self._is_file_in_date_range(filename):
                            continue
                        
                        remote_path = f"{year_path}/{filename}"
                        local_path = os.path.join(
                            self.config.local_base_dir,
                            str(year),
                            filename
                        )
                        
                        # Get file size
                        try:
                            size = ftp.size(filename)
                        except Exception:
                            size = 0
                        
                        # Extract month for FileInfo
                        result = self._extract_yearmonth_from_filename(filename)
                        month_int = result[1] if result else 0
                        
                        files.append(FileInfo(
                            remote_path=remote_path,
                            local_path=local_path,
                            size=size or 0,
                            year=year,
                            month=month_int
                        ))
                        files_in_year += 1
                
                if files_in_year > 0:
                    logger.info(f"  Found {files_in_year} files in {year}")
                    
        finally:
            self.connection_pool.return_connection(ftp)
        
        logger.info(f"✓ Total Envisat L3C files discovered: {len(files)}")
        total_size_gb = sum(f.size for f in files) / (1024**3)
        logger.info(f"✓ Total size: {total_size_gb:.2f} GB")
        
        return files
    
    def _download_file(self, file_info: FileInfo) -> Tuple[bool, str]:
        """Download a single file with retry logic."""
        # Check if file already exists and is complete
        if os.path.exists(file_info.local_path):
            local_size = os.path.getsize(file_info.local_path)
            if file_info.size > 0 and local_size >= file_info.size:
                return True, "skipped"
        
        # Create directory if needed
        os.makedirs(os.path.dirname(file_info.local_path), exist_ok=True)
        
        for attempt in range(self.config.max_retries):
            ftp = None
            try:
                ftp = self.connection_pool.get_connection()
                
                # Change to the correct directory
                ftp.cwd(self.config.ftp_base_path)
                
                # Download file
                with open(file_info.local_path, 'wb') as f:
                    ftp.retrbinary(
                        f'RETR {file_info.remote_path}',
                        f.write,
                        blocksize=self.config.chunk_size
                    )
                
                self.connection_pool.return_connection(ftp)
                return True, "downloaded"
                
            except Exception as e:
                if ftp:
                    try:
                        ftp.quit()
                    except:
                        pass
                
                if attempt < self.config.max_retries - 1:
                    time.sleep(self.config.retry_delay)
                else:
                    # Clean up partial file
                    if os.path.exists(file_info.local_path):
                        try:
                            os.remove(file_info.local_path)
                        except:
                            pass
                    return False, str(e)
        
        return False, "Max retries exceeded"
    
    def _update_stats(self, file_info: FileInfo, success: bool, status: str):
        """Thread-safe stats update."""
        with self._stats_lock:
            if success:
                if status == "skipped":
                    self.stats.skipped_files += 1
                else:
                    self.stats.completed_files += 1
                    self.stats.downloaded_bytes += file_info.size
            else:
                self.stats.failed_files += 1
                self.stats.failed_list.append(file_info.remote_path)
    
    def download_all(self, files: List[FileInfo], show_progress_interval: int = 10):
        """Download all files using thread pool."""
        self.stats = DownloadStats(
            total_files=len(files),
            total_bytes=sum(f.size for f in files),
            start_time=datetime.now()
        )
        
        logger.info(f"Starting download of {len(files)} Envisat L3C files with {self.config.max_workers} workers...")
        
        last_progress_time = time.time()
        
        with ThreadPoolExecutor(max_workers=self.config.max_workers) as executor:
            # Submit all download tasks
            future_to_file = {
                executor.submit(self._download_file, f): f 
                for f in files
            }
            
            for future in as_completed(future_to_file):
                file_info = future_to_file[future]
                try:
                    success, status = future.result()
                    self._update_stats(file_info, success, status)
                except Exception as e:
                    self._update_stats(file_info, False, str(e))
                
                # Progress report
                current_time = time.time()
                if current_time - last_progress_time >= show_progress_interval:
                    self._print_progress()
                    last_progress_time = current_time
        
        self._print_final_stats()
    
    def _print_progress(self):
        """Print current progress."""
        with self._stats_lock:
            progress = self.stats.get_progress_percent()
            speed = self.stats.get_speed_mbps()
            eta = self.stats.get_eta()
            
            print(f"\r📊 Progress: {progress:.1f}% | "
                  f"Files: {self.stats.completed_files + self.stats.skipped_files}/{self.stats.total_files} | "
                  f"Speed: {speed:.2f} MB/s | "
                  f"ETA: {eta} | "
                  f"Skipped: {self.stats.skipped_files} | "
                  f"Failed: {self.stats.failed_files}", end="")
            print()  # New line for notebook
    
    def _print_final_stats(self):
        """Print final download statistics."""
        elapsed = (datetime.now() - self.stats.start_time).total_seconds()
        
        print("\n" + "=" * 60)
        print("ENVISAT L3C DOWNLOAD COMPLETE")
        print("=" * 60)
        print(f"✓ Total files processed: {self.stats.total_files}")
        print(f"✓ Successfully downloaded: {self.stats.completed_files}")
        print(f"✓ Skipped (already exists): {self.stats.skipped_files}")
        print(f"✗ Failed: {self.stats.failed_files}")
        print(f"📦 Total data downloaded: {self.stats.downloaded_bytes / (1024**3):.2f} GB")
        print(f"⏱ Total time: {timedelta(seconds=int(elapsed))}")
        print(f"🚀 Average speed: {self.stats.get_speed_mbps():.2f} MB/s")
        print("=" * 60)
        
        if self.stats.failed_list:
            print("\n⚠ Failed files:")
            for f in self.stats.failed_list[:10]:
                print(f"  - {f}")
            if len(self.stats.failed_list) > 10:
                print(f"  ... and {len(self.stats.failed_list) - 10} more")

print("Envisat L3C Download Manager class defined")

Envisat L3C Download Manager class defined


In [12]:
# =============================================================================
# ENVISAT L3C MAIN EXECUTION
# =============================================================================

def main_env_l3c():
    """Main execution function for Envisat L3C downloads."""
    
    print("=" * 60)
    print("CCI V4 ENVISAT L3C DATA DOWNLOAD")
    print("=" * 60)
    print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # Check and create local directory
    local_dir = Path(env_l3c_config.local_base_dir)
    if not local_dir.exists():
        local_dir.mkdir(parents=True, exist_ok=True)
        print(f"✓ Created directory: {local_dir}")
    else:
        print(f"✓ Directory exists: {local_dir}")
    
    # Initial memory check
    print("\n--- Initial Memory Status ---")
    memory_monitor.print_memory_status()
    
    # Start memory monitoring
    memory_monitor.start_monitoring(interval=60)
    
    # Initialize download manager
    manager = ENVL3CDownloadManager(env_l3c_config)
    
    try:
        # Connect to FTP
        manager.connect()
        
        # Discover files
        print("\n--- File Discovery ---")
        files = manager.discover_files()
        
        if not files:
            print("No Envisat L3C files found to download!")
            return
        
        # Confirm download
        total_size_gb = sum(f.size for f in files) / (1024**3)
        print(f"\n📁 Ready to download {len(files)} Envisat L3C files ({total_size_gb:.2f} GB)")
        print(f"📂 Destination: {env_l3c_config.local_base_dir}")
        
        # Memory check before download
        print("\n--- Pre-Download Memory Status ---")
        stats = memory_monitor.print_memory_status()
        
        if stats['percent_used'] > env_l3c_config.memory_critical_threshold:
            print("⚠ WARNING: Memory usage is critical! Consider freeing memory before proceeding.")
        
        # Start download
        print("\n--- Starting Envisat L3C Download ---")
        manager.download_all(files)
        
        # Final memory status
        print("\n--- Final Memory Status ---")
        memory_monitor.print_memory_status()
        
    except KeyboardInterrupt:
        print("\n\n⚠ Download interrupted by user!")
    except Exception as e:
        logger.error(f"Error during Envisat L3C download: {e}")
        raise
    finally:
        # Cleanup
        manager.disconnect()
        memory_monitor.stop_monitoring()
        print(f"\nEnd Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Run the Envisat L3C download
if __name__ == "__main__" or True:  # True for notebook execution
    main_env_l3c()

2025-12-07 14:40:46 - INFO - Memory monitoring started
2025-12-07 14:40:46 - INFO - Connecting to ftp.awi.de...


CCI V4 ENVISAT L3C DATA DOWNLOAD
Start Time: 2025-12-07 14:40:46

✓ Directory exists: D:\phd\data\CCI_v4\l3c\ENV

--- Initial Memory Status ---
✓ Memory: 10.1GB used / 15.4GB total (65.4% used) | Available: 5.3GB (34.6%)


2025-12-07 14:40:48 - INFO - ✓ Connected successfully to Envisat L3C directory!
2025-12-07 14:40:48 - INFO - Discovering Envisat L3C files on FTP server...
2025-12-07 14:40:48 - INFO - Date filter: 2002-01 to 2012-12



--- File Discovery ---


2025-12-07 14:40:51 - INFO -   Found 3 files in 2002
2025-12-07 14:40:56 - INFO -   Found 12 files in 2003
2025-12-07 14:41:01 - INFO -   Found 12 files in 2004
2025-12-07 14:41:06 - INFO -   Found 12 files in 2005
2025-12-07 14:41:11 - INFO -   Found 12 files in 2006
2025-12-07 14:41:16 - INFO -   Found 12 files in 2007
2025-12-07 14:41:21 - INFO -   Found 12 files in 2008
2025-12-07 14:41:26 - INFO -   Found 12 files in 2009
2025-12-07 14:41:31 - INFO -   Found 12 files in 2010
2025-12-07 14:41:36 - INFO -   Found 12 files in 2011
2025-12-07 14:41:39 - INFO -   Found 3 files in 2012
2025-12-07 14:41:39 - INFO - ✓ Total Envisat L3C files discovered: 114
2025-12-07 14:41:39 - INFO - ✓ Total size: 0.08 GB
2025-12-07 14:41:39 - INFO - Starting download of 114 Envisat L3C files with 8 workers...



📁 Ready to download 114 Envisat L3C files (0.08 GB)
📂 Destination: D:\phd\data\CCI_v4\l3c\ENV

--- Pre-Download Memory Status ---
✓ Memory: 10.1GB used / 15.4GB total (65.7% used) | Available: 5.3GB (34.3%)

--- Starting Envisat L3C Download ---
📊 Progress: 6.1% | Files: 7/114 | Speed: 0.47 MB/s | ETA: 0:02:41 | Skipped: 0 | Failed: 0
📊 Progress: 23.7% | Files: 27/114 | Speed: 0.96 MB/s | ETA: 0:01:06 | Skipped: 0 | Failed: 0
📊 Progress: 43.0% | Files: 49/114 | Speed: 1.14 MB/s | ETA: 0:00:41 | Skipped: 0 | Failed: 0
📊 Progress: 61.4% | Files: 70/114 | Speed: 1.22 MB/s | ETA: 0:00:26 | Skipped: 0 | Failed: 0
📊 Progress: 79.8% | Files: 91/114 | Speed: 1.28 MB/s | ETA: 0:00:13 | Skipped: 0 | Failed: 0
📊 Progress: 99.1% | Files: 113/114 | Speed: 1.33 MB/s | ETA: 0:00:00 | Skipped: 0 | Failed: 0

ENVISAT L3C DOWNLOAD COMPLETE
✓ Total files processed: 114
✓ Successfully downloaded: 114
✓ Skipped (already exists): 0
✗ Failed: 0
📦 Total data downloaded: 0.08 GB
⏱ Total time: 0:01:02
🚀 Averag

2025-12-07 14:42:44 - INFO - All connections closed
2025-12-07 14:42:44 - INFO - Memory monitoring stopped



End Time: 2025-12-07 14:42:44


# ESA CCI Sea Ice Version 4 - CryoSat-2 L3C Data Download

## Dataset Information
- **Product**: ESA Sea Ice Climate Change Initiative (Sea_Ice_cci)
- **Description**: Southern hemisphere sea ice thickness from CryoSat-2 on 50km EASE2 grid (L3C), v4.0
- **FTP Source**: `ftp://ftp.awi.de/sea_ice/projects/cci/crdp/v4p0/l3c_release/sh/cryosat2/`
- **Time Range**: 2010 - 2024
- **Local Destination**: `D:\phd\data\CCI_v4\l3c\CS2`
- **File Pattern**: `ESACCI-SEAICE-L3C-SITHICK-SIRAL_CRYOSAT2-SH_50KM_EASE2-YYYYMM-fv4p0.nc`
- **Temporal Resolution**: Monthly

In [13]:
# =============================================================================
# CRYOSAT-2 L3C CONFIGURATION
# =============================================================================

@dataclass
class CS2L3CDownloadConfig:
    """Configuration for the CryoSat-2 L3C FTP download manager."""
    
    # FTP Settings
    ftp_host: str = "ftp.awi.de"
    ftp_base_path: str = "/sea_ice/projects/cci/crdp/v4p0/l3c_release/sh/cryosat2"
    ftp_timeout: int = 60  # seconds
    
    # Local Settings
    local_base_dir: str = r"D:\phd\data\CCI_v4\l3c\CS2"
    
    # Date Range: Full CryoSat-2 L3C record (2010 - 2024)
    start_year: int = 2010
    start_month: int = 1
    end_year: int = 2024
    end_month: int = 12
    
    # Performance Settings
    max_workers: int = 8  # Concurrent download threads
    max_retries: int = 3
    retry_delay: int = 5  # seconds
    chunk_size: int = 1024 * 1024  # 1MB chunks for download
    
    # Memory Thresholds (percentage)
    memory_warning_threshold: float = 80.0
    memory_critical_threshold: float = 90.0
    
    # Connection Pool
    connection_pool_size: int = 10
    
    def get_start_date(self) -> str:
        """Get the start date as YYYY-MM string."""
        return f"{self.start_year}-{self.start_month:02d}"
    
    def get_end_date(self) -> str:
        """Get the end date as YYYY-MM string."""
        return f"{self.end_year}-{self.end_month:02d}"

cs2_l3c_config = CS2L3CDownloadConfig()

print("=" * 60)
print("CRYOSAT-2 L3C DOWNLOAD CONFIGURATION")
print("=" * 60)
print(f"FTP Host: {cs2_l3c_config.ftp_host}")
print(f"FTP Path: {cs2_l3c_config.ftp_base_path}")
print(f"Local Directory: {cs2_l3c_config.local_base_dir}")
print(f"Date Range: {cs2_l3c_config.get_start_date()} to {cs2_l3c_config.get_end_date()}")
print(f"Concurrent Workers: {cs2_l3c_config.max_workers}")
print(f"Max Retries: {cs2_l3c_config.max_retries}")
print("=" * 60)

CRYOSAT-2 L3C DOWNLOAD CONFIGURATION
FTP Host: ftp.awi.de
FTP Path: /sea_ice/projects/cci/crdp/v4p0/l3c_release/sh/cryosat2
Local Directory: D:\phd\data\CCI_v4\l3c\CS2
Date Range: 2010-01 to 2024-12
Concurrent Workers: 8
Max Retries: 3


In [14]:
class CS2L3CDownloadManager:
    """Manager for downloading CCI CryoSat-2 L3C datasets (monthly gridded)."""
    
    def __init__(self, config: CS2L3CDownloadConfig):
        self.config = config
        self.connection_pool: Optional[FTPConnectionPool] = None
        self.stats = DownloadStats()
        self._stats_lock = threading.Lock()
        
    def connect(self):
        """Initialize connection pool."""
        logger.info(f"Connecting to {self.config.ftp_host}...")
        self.connection_pool = FTPConnectionPool(
            host=self.config.ftp_host,
            base_path=self.config.ftp_base_path,
            pool_size=self.config.connection_pool_size,
            timeout=self.config.ftp_timeout
        )
        # Test connection
        ftp = self.connection_pool.get_connection()
        self.connection_pool.return_connection(ftp)
        logger.info("✓ Connected successfully to CryoSat-2 L3C directory!")
        
    def disconnect(self):
        """Close all connections."""
        if self.connection_pool:
            self.connection_pool.close_all()
            logger.info("All connections closed")
    
    def _extract_yearmonth_from_filename(self, filename: str) -> Optional[Tuple[int, int]]:
        """Extract year and month from filename like ESACCI-SEAICE-L3C-SITHICK-SIRAL_CRYOSAT2-SH_50KM_EASE2-201011-fv4p0.nc"""
        try:
            import re
            # Match 6-digit YYYYMM pattern before -fv
            match = re.search(r'-(\d{6})-fv', filename)
            if match:
                yyyymm = match.group(1)
                year = int(yyyymm[:4])
                month = int(yyyymm[4:6])
                return (year, month)
        except:
            pass
        return None
    
    def _is_file_in_date_range(self, filename: str) -> bool:
        """Check if file year/month is within configured date range."""
        result = self._extract_yearmonth_from_filename(filename)
        if result is None:
            return False
        
        year, month = result
        
        # Check if within range
        start_val = self.config.start_year * 100 + self.config.start_month
        end_val = self.config.end_year * 100 + self.config.end_month
        file_val = year * 100 + month
        
        return start_val <= file_val <= end_val
    
    def discover_files(self) -> List[FileInfo]:
        """Discover all CryoSat-2 L3C files to download within the specified date range."""
        logger.info("Discovering CryoSat-2 L3C files on FTP server...")
        logger.info(f"Date filter: {self.config.get_start_date()} to {self.config.get_end_date()}")
        files = []
        
        ftp = self.connection_pool.get_connection()
        try:
            for year in range(self.config.start_year, self.config.end_year + 1):
                year_path = str(year)
                
                try:
                    ftp.cwd(f"{self.config.ftp_base_path}/{year_path}")
                except ftplib.error_perm:
                    logger.warning(f"Year {year} directory not found, skipping...")
                    continue
                
                # List files directly in year directory (L3C has monthly files per year)
                year_files = []
                ftp.retrlines('NLST', year_files.append)
                
                files_in_year = 0
                for filename in year_files:
                    if filename.endswith('.nc'):
                        # Apply date filter
                        if not self._is_file_in_date_range(filename):
                            continue
                        
                        remote_path = f"{year_path}/{filename}"
                        local_path = os.path.join(
                            self.config.local_base_dir,
                            str(year),
                            filename
                        )
                        
                        # Get file size
                        try:
                            size = ftp.size(filename)
                        except Exception:
                            size = 0
                        
                        # Extract month for FileInfo
                        result = self._extract_yearmonth_from_filename(filename)
                        month_int = result[1] if result else 0
                        
                        files.append(FileInfo(
                            remote_path=remote_path,
                            local_path=local_path,
                            size=size or 0,
                            year=year,
                            month=month_int
                        ))
                        files_in_year += 1
                
                if files_in_year > 0:
                    logger.info(f"  Found {files_in_year} files in {year}")
                    
        finally:
            self.connection_pool.return_connection(ftp)
        
        logger.info(f"✓ Total CryoSat-2 L3C files discovered: {len(files)}")
        total_size_gb = sum(f.size for f in files) / (1024**3)
        logger.info(f"✓ Total size: {total_size_gb:.2f} GB")
        
        return files
    
    def _download_file(self, file_info: FileInfo) -> Tuple[bool, str]:
        """Download a single file with retry logic."""
        # Check if file already exists and is complete
        if os.path.exists(file_info.local_path):
            local_size = os.path.getsize(file_info.local_path)
            if file_info.size > 0 and local_size >= file_info.size:
                return True, "skipped"
        
        # Create directory if needed
        os.makedirs(os.path.dirname(file_info.local_path), exist_ok=True)
        
        for attempt in range(self.config.max_retries):
            ftp = None
            try:
                ftp = self.connection_pool.get_connection()
                
                # Change to the correct directory
                ftp.cwd(self.config.ftp_base_path)
                
                # Download file
                with open(file_info.local_path, 'wb') as f:
                    ftp.retrbinary(
                        f'RETR {file_info.remote_path}',
                        f.write,
                        blocksize=self.config.chunk_size
                    )
                
                self.connection_pool.return_connection(ftp)
                return True, "downloaded"
                
            except Exception as e:
                if ftp:
                    try:
                        ftp.quit()
                    except:
                        pass
                
                if attempt < self.config.max_retries - 1:
                    time.sleep(self.config.retry_delay)
                else:
                    # Clean up partial file
                    if os.path.exists(file_info.local_path):
                        try:
                            os.remove(file_info.local_path)
                        except:
                            pass
                    return False, str(e)
        
        return False, "Max retries exceeded"
    
    def _update_stats(self, file_info: FileInfo, success: bool, status: str):
        """Thread-safe stats update."""
        with self._stats_lock:
            if success:
                if status == "skipped":
                    self.stats.skipped_files += 1
                else:
                    self.stats.completed_files += 1
                    self.stats.downloaded_bytes += file_info.size
            else:
                self.stats.failed_files += 1
                self.stats.failed_list.append(file_info.remote_path)
    
    def download_all(self, files: List[FileInfo], show_progress_interval: int = 10):
        """Download all files using thread pool."""
        self.stats = DownloadStats(
            total_files=len(files),
            total_bytes=sum(f.size for f in files),
            start_time=datetime.now()
        )
        
        logger.info(f"Starting download of {len(files)} CryoSat-2 L3C files with {self.config.max_workers} workers...")
        
        last_progress_time = time.time()
        
        with ThreadPoolExecutor(max_workers=self.config.max_workers) as executor:
            # Submit all download tasks
            future_to_file = {
                executor.submit(self._download_file, f): f 
                for f in files
            }
            
            for future in as_completed(future_to_file):
                file_info = future_to_file[future]
                try:
                    success, status = future.result()
                    self._update_stats(file_info, success, status)
                except Exception as e:
                    self._update_stats(file_info, False, str(e))
                
                # Progress report
                current_time = time.time()
                if current_time - last_progress_time >= show_progress_interval:
                    self._print_progress()
                    last_progress_time = current_time
        
        self._print_final_stats()
    
    def _print_progress(self):
        """Print current progress."""
        with self._stats_lock:
            progress = self.stats.get_progress_percent()
            speed = self.stats.get_speed_mbps()
            eta = self.stats.get_eta()
            
            print(f"\r📊 Progress: {progress:.1f}% | "
                  f"Files: {self.stats.completed_files + self.stats.skipped_files}/{self.stats.total_files} | "
                  f"Speed: {speed:.2f} MB/s | "
                  f"ETA: {eta} | "
                  f"Skipped: {self.stats.skipped_files} | "
                  f"Failed: {self.stats.failed_files}", end="")
            print()  # New line for notebook
    
    def _print_final_stats(self):
        """Print final download statistics."""
        elapsed = (datetime.now() - self.stats.start_time).total_seconds()
        
        print("\n" + "=" * 60)
        print("CRYOSAT-2 L3C DOWNLOAD COMPLETE")
        print("=" * 60)
        print(f"✓ Total files processed: {self.stats.total_files}")
        print(f"✓ Successfully downloaded: {self.stats.completed_files}")
        print(f"✓ Skipped (already exists): {self.stats.skipped_files}")
        print(f"✗ Failed: {self.stats.failed_files}")
        print(f"📦 Total data downloaded: {self.stats.downloaded_bytes / (1024**3):.2f} GB")
        print(f"⏱ Total time: {timedelta(seconds=int(elapsed))}")
        print(f"🚀 Average speed: {self.stats.get_speed_mbps():.2f} MB/s")
        print("=" * 60)
        
        if self.stats.failed_list:
            print("\n⚠ Failed files:")
            for f in self.stats.failed_list[:10]:
                print(f"  - {f}")
            if len(self.stats.failed_list) > 10:
                print(f"  ... and {len(self.stats.failed_list) - 10} more")

print("CryoSat-2 L3C Download Manager class defined")

CryoSat-2 L3C Download Manager class defined


In [15]:
# =============================================================================
# CRYOSAT-2 L3C MAIN EXECUTION
# =============================================================================

def main_cs2_l3c():
    """Main execution function for CryoSat-2 L3C downloads."""
    
    print("=" * 60)
    print("CCI V4 CRYOSAT-2 L3C DATA DOWNLOAD")
    print("=" * 60)
    print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # Check and create local directory
    local_dir = Path(cs2_l3c_config.local_base_dir)
    if not local_dir.exists():
        local_dir.mkdir(parents=True, exist_ok=True)
        print(f"✓ Created directory: {local_dir}")
    else:
        print(f"✓ Directory exists: {local_dir}")
    
    # Initial memory check
    print("\n--- Initial Memory Status ---")
    memory_monitor.print_memory_status()
    
    # Start memory monitoring
    memory_monitor.start_monitoring(interval=60)
    
    # Initialize download manager
    manager = CS2L3CDownloadManager(cs2_l3c_config)
    
    try:
        # Connect to FTP
        manager.connect()
        
        # Discover files
        print("\n--- File Discovery ---")
        files = manager.discover_files()
        
        if not files:
            print("No CryoSat-2 L3C files found to download!")
            return
        
        # Confirm download
        total_size_gb = sum(f.size for f in files) / (1024**3)
        print(f"\n📁 Ready to download {len(files)} CryoSat-2 L3C files ({total_size_gb:.2f} GB)")
        print(f"📂 Destination: {cs2_l3c_config.local_base_dir}")
        
        # Memory check before download
        print("\n--- Pre-Download Memory Status ---")
        stats = memory_monitor.print_memory_status()
        
        if stats['percent_used'] > cs2_l3c_config.memory_critical_threshold:
            print("⚠ WARNING: Memory usage is critical! Consider freeing memory before proceeding.")
        
        # Start download
        print("\n--- Starting CryoSat-2 L3C Download ---")
        manager.download_all(files)
        
        # Final memory status
        print("\n--- Final Memory Status ---")
        memory_monitor.print_memory_status()
        
    except KeyboardInterrupt:
        print("\n\n⚠ Download interrupted by user!")
    except Exception as e:
        logger.error(f"Error during CryoSat-2 L3C download: {e}")
        raise
    finally:
        # Cleanup
        manager.disconnect()
        memory_monitor.stop_monitoring()
        print(f"\nEnd Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Run the CryoSat-2 L3C download
if __name__ == "__main__" or True:  # True for notebook execution
    main_cs2_l3c()

2025-12-07 14:48:23 - INFO - Memory monitoring started
2025-12-07 14:48:23 - INFO - Connecting to ftp.awi.de...


CCI V4 CRYOSAT-2 L3C DATA DOWNLOAD
Start Time: 2025-12-07 14:48:23

✓ Directory exists: D:\phd\data\CCI_v4\l3c\CS2

--- Initial Memory Status ---
✓ Memory: 10.1GB used / 15.4GB total (65.3% used) | Available: 5.4GB (34.7%)


2025-12-07 14:48:25 - INFO - ✓ Connected successfully to CryoSat-2 L3C directory!
2025-12-07 14:48:25 - INFO - Discovering CryoSat-2 L3C files on FTP server...
2025-12-07 14:48:25 - INFO - Date filter: 2010-01 to 2024-12



--- File Discovery ---


2025-12-07 14:48:27 - INFO -   Found 2 files in 2010
2025-12-07 14:48:32 - INFO -   Found 12 files in 2011
2025-12-07 14:48:38 - INFO -   Found 12 files in 2012
2025-12-07 14:48:43 - INFO -   Found 12 files in 2013
2025-12-07 14:48:48 - INFO -   Found 12 files in 2014
2025-12-07 14:48:53 - INFO -   Found 12 files in 2015
2025-12-07 14:48:58 - INFO -   Found 12 files in 2016
2025-12-07 14:49:03 - INFO -   Found 12 files in 2017
2025-12-07 14:49:09 - INFO -   Found 12 files in 2018
2025-12-07 14:49:14 - INFO -   Found 12 files in 2019
2025-12-07 14:49:19 - INFO -   Found 12 files in 2020
2025-12-07 14:49:24 - INFO -   Found 12 files in 2021
2025-12-07 14:49:29 - INFO -   Found 12 files in 2022
2025-12-07 14:49:34 - INFO -   Found 12 files in 2023
2025-12-07 14:49:37 - INFO -   Found 4 files in 2024
2025-12-07 14:49:38 - INFO - ✓ Total CryoSat-2 L3C files discovered: 162
2025-12-07 14:49:38 - INFO - ✓ Total size: 0.11 GB
2025-12-07 14:49:38 - INFO - Starting download of 162 CryoSat-2 L3C 


📁 Ready to download 162 CryoSat-2 L3C files (0.11 GB)
📂 Destination: D:\phd\data\CCI_v4\l3c\CS2

--- Pre-Download Memory Status ---
✓ Memory: 9.9GB used / 15.4GB total (63.9% used) | Available: 5.6GB (36.1%)

--- Starting CryoSat-2 L3C Download ---
📊 Progress: 4.3% | Files: 7/162 | Speed: 0.45 MB/s | ETA: 0:04:01 | Skipped: 0 | Failed: 0
📊 Progress: 16.7% | Files: 27/162 | Speed: 0.93 MB/s | ETA: 0:01:45 | Skipped: 0 | Failed: 0
📊 Progress: 30.2% | Files: 49/162 | Speed: 1.14 MB/s | ETA: 0:01:11 | Skipped: 0 | Failed: 0
📊 Progress: 43.2% | Files: 70/162 | Speed: 1.23 MB/s | ETA: 0:00:54 | Skipped: 0 | Failed: 0
📊 Progress: 56.2% | Files: 91/162 | Speed: 1.27 MB/s | ETA: 0:00:40 | Skipped: 0 | Failed: 0
📊 Progress: 69.1% | Files: 112/162 | Speed: 1.30 MB/s | ETA: 0:00:27 | Skipped: 0 | Failed: 0
📊 Progress: 82.1% | Files: 133/162 | Speed: 1.33 MB/s | ETA: 0:00:15 | Skipped: 0 | Failed: 0
📊 Progress: 95.1% | Files: 154/162 | Speed: 1.35 MB/s | ETA: 0:00:04 | Skipped: 0 | Failed: 0

CRYO

2025-12-07 14:51:06 - INFO - All connections closed
2025-12-07 14:51:06 - INFO - Memory monitoring stopped



End Time: 2025-12-07 14:51:06
